In [3]:
import tensorflow as tf
import tensorflow_probability as tfp
import tensorflow_hub as hub 
from data import galsim

import sys
sys.path.insert(0, 'pixel-cnn/')

In [2]:
data_dir='/usr/local/share/galsim/COSMOS_25.2_training_sample'
batch_size=32

# Let's build an input function for the model
input_fn = galsim.build_input_pipeline(data_dir,
                                        batch_size=batch_size,
                                        stamp_size=64,
                                        pixel_size=0.03,
                                        input_nprocs=12,
                                        nrepeat=10,
                                        cache_dir='/data2/COSMOS/cache64_10_hst')

/usr/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216, got 192
  return f(*args, **kwds)
/usr/lib/python3.7/importlib/_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
/usr/lib/python3.7/importlib/_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
/usr/lib/python3.7/importlib/_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
/usr/lib/python3.7/importlib/_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
/usr/lib/python3.7/importlib/_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __packa

In [5]:
# Build a pixelCNN Estimator
from pixel_cnn_pp.model import model_spec

In [6]:
def pack_images(images, rows, cols):
    """Helper utility to make a field of images."""
    shape = tf.shape(images)
    width = shape[-3]
    height = shape[-2]
    depth = shape[-1]
    images = tf.reshape(images, (-1, width, height, depth))
    batch = tf.shape(images)[0]
    rows = tf.minimum(rows, batch)
    cols = tf.minimum(batch // rows, cols)
    images = images[:rows * cols]
    images = tf.reshape(images, (rows, cols, width, height, depth))
    images = tf.transpose(images, [0, 2, 1, 3, 4])
    images = tf.reshape(images, [1, rows * width, cols * height, depth])
    return images


def image_tile_summary(name, tensor, rows=8, cols=8):
    tf.summary.image(name, pack_images(tensor, rows, cols), max_outputs=1)

In [7]:
n_channels=1
image_size=64

def pixelcnn_model_fn(features, labels, mode, params, config):
    is_training = (mode == tf.estimator.ModeKeys.TRAIN)

    # Extract input images
    x = features['x']
    
    model_opt = { 'nr_resnet': 2, 'nr_filters': 64, 'nr_logistic_mix': 1, 'resnet_nonlinearity': 'concat_elu', 'energy_distance': False}
    
    if mode == tf.estimator.ModeKeys.PREDICT:
        # Build the model
        def make_model_spec():
            input_layer = tf.placeholder(tf.float32, shape=[1,image_size, image_size,1])
            model = tf.make_template('model', model_spec)
            out = model(input_layer, None, ema=None, dropout_p=0., **model_opt)
            loc, scale = tf.split(out, num_or_size_splits=2,axis=-1)
            scale = tf.nn.softplus(scale) + 1e-4
            distribution = tfp.distributions.Independent( tfp.distributions.Normal(loc=loc, scale=scale))
            log_prob = - distribution.log_prob(input_layer)
            print(log_prob)
            grads = tf.gradients(log_prob[0], input_layer)
            samples = distribution.sample()
            hub.add_signature(inputs=input_layer,
                              outputs=grads[0])#{'grads':grads[0], 'sample':samples,'log_prob':log_prob})
        
        spec = hub.create_module_spec(make_model_spec, drop_collections=['checkpoints'])
        pixelcnn = hub.Module(spec, name="pixelcnn_module")
            
        hub.register_module_for_export(pixelcnn, "pixelcnn_gradients")
        predictions =  pixelcnn(x, as_dict=True)
        return tf.estimator.EstimatorSpec(mode=mode,
                                          predictions=predictions)
    
    # Build the model
    def make_model_spec():
        input_layer = tf.placeholder(tf.float32, shape=[batch_size, image_size, image_size, n_channels])
        
        model = tf.make_template('model', model_spec)
        out = model(input_layer, None, ema=None, dropout_p=0.5, **model_opt)
        out = tf.layers.dense(out, 2, activation=None) # project the output to only 2 values
        loc, scale = tf.split(out, num_or_size_splits=2,axis=-1)
        scale = tf.nn.softplus(scale) + 1e-4
        print(loc, scale)
        # Build Gaussian model for output value
        distribution = tfp.distributions.Independent( tfp.distributions.Normal(loc=loc, scale=scale))
        sample = distribution.sample()
        log_prob = distribution.log_prob(input_layer)
        hub.add_signature(inputs=input_layer,
                          outputs={'sample': sample, 'log_prob': log_prob})

    spec = hub.create_module_spec(make_model_spec, drop_collections=['checkpoints'])
    pixelcnn = hub.Module(spec, name="pixelcnn_module", trainable=True)

    output = pixelcnn(x, as_dict=True)

    loglikelihood = output['log_prob']
    sample = output['sample']
    print(loglikelihood)
    print(sample)
    tf.summary.scalar('loglikelihood', tf.reduce_mean(loglikelihood))

    image_tile_summary("image", tf.to_float(x[:16]), rows=4, cols=4)
    image_tile_summary("recon", tf.to_float(sample[:16]), rows=4, cols=4)

    loss = -tf.reduce_mean(loglikelihood)

    # Training of the model
    global_step = tf.train.get_or_create_global_step()
    learning_rate = tf.train.cosine_decay(params["learning_rate"], global_step,
                                          params["max_steps"])
    tf.summary.scalar("learning_rate", learning_rate)
    optimizer = tf.train.AdamOptimizer(learning_rate)

    update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
    with tf.control_dependencies(update_ops):
        train_op = optimizer.minimize(loss , global_step=global_step)

    eval_metric_ops = {"loglikelihood": tf.metrics.mean(tf.reduce_mean(loglikelihood))}

    return tf.estimator.EstimatorSpec(mode=mode,
                                      loss=loss,
                                      train_op=train_op,
                                      eval_metric_ops=eval_metric_ops)

In [8]:
params={'learning_rate':0.0002, 'max_steps':200000}

In [9]:
mymodel = tf.estimator.Estimator(model_fn=pixelcnn_model_fn,
                                params=params, model_dir='/data2/cosmos_pixel_cnn_hst')

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/data2/cosmos_pixel_cnn_hst', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fe366d50320>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [ ]:
mymodel.train(input_fn, steps=100000)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
tf.py_func is deprecated in TF V2. Instead, use
    tf.py_function, which takes a python function which manipulates tf eager
    tensors instead of numpy arrays. It's easy to convert a tf eager tensor to
    an ndarray (just call tensor.numpy()) but having access to eager tensors
    means `tf.py_function`s can use accelerators such as GPUs as well as
    being differentiable using a gradient tape.
    
INFO:tensorflow:Calling model_fn.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


/usr/lib/python3.7/site-packages/numpy/lib/type_check.py:546: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  'a.item() instead', DeprecationWarning, stacklevel=1)
/usr/lib/python3.7/site-packages/numpy/lib/type_check.py:546: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  'a.item() instead', DeprecationWarning, stacklevel=1)


Instructions for updating:
Use keras.layers.dense instead.
Tensor("split:0", shape=(32, 64, 64, 1), dtype=float32) Tensor("add:0", shape=(32, 64, 64, 1), dtype=float32)
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
Tensor("pixelcnn_module_apply_default/IndependentNormal/log_prob/Sum:0", shape=(32,), dtype=float32)
Tensor("pixelcnn_module_apply_default/IndependentNormal/sample/Reshape:0", shape=(32, 64, 64, 1), dtype=float32)
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use tf.cast instead.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkp

In [ ]:
# Code to export the trained model:
exporter = hub.LatestModuleExporter("tf_hub",
        tf.estimator.export.build_raw_serving_input_receiver_fn(input_fn()[0]))

exporter.export(mymodel, '/data2/cosmos_pixel_cnn_hst/', mymodel.latest_checkpoint())